In [ ]:
import gensim
import string
import os
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [ ]:
bbc_news = pd.read_csv('208_data/bbc_news.csv')

In [ ]:
concat_data = bbc_news['title'] + " " +bbc_news['description']
article_list = concat_data.tolist()


In [ ]:
tokenizer = nltk.tokenize #.split() is better lol
lemma = WordNetLemmatizer()
swords = stopwords.words("english")


new_list = []

for article in article_list:
    
    clean_article = []
    
    #tokenize the single string article into an array of strings for each word
    tokenized_article = article.lower().split()
    
    for word in tokenized_article:
        if word not in swords and word not in string.punctuation: #remove stopwords and punctuation
            word = re.sub("[^a-zA-Z0-9]","",word) #replace all puctuation within a string with ''  "world's" -> "worlds"
            word = lemma.lemmatize(word) #lemmatize word
            word =  re.sub(r"\b[a-zA-Z]\b", "", word)
            
            #SPECIAL CASES
            if word == "russian": word = "russia"
            if word == "ukrainian": word = "ukraine"
            if word == "says": word = "say"
            
            if word != "say":
                clean_article.append(word)
    
    
    
    new_list.append(clean_article)

In [ ]:
texts = new_list
# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1


texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

In [ ]:
new_list = []
for text in texts: 
  joined = ' '.join(text)
  new_list.append(joined)

In [ ]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic

In [ ]:
!pip install tbb


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 5.3 MB/s 


In [ ]:

from umap import UMAP

In [ ]:
# Prepare embeddings
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(new_list, show_progress_bar=False)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
# Set the random state in the UMAP model to prevent stochastic behavior 
umap_model = UMAP(n_neighbors=15, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=42)

In [ ]:
# Train model
topic_model = BERTopic(umap_model=umap_model, verbose=True, nr_topics=15)
topics, probs = topic_model.fit_transform(new_list, embeddings)

2022-06-01 06:03:17,320 - BERTopic - Reduced dimensionality
2022-06-01 06:03:17,477 - BERTopic - Clustered reduced embeddings
2022-06-01 06:03:18,067 - BERTopic - Reduced number of topics from 74 to 16


In [ ]:
# Transform same documents and check if the predicted topics are the same
new_topics, new_probs = topic_model.transform(new_list, embeddings)
assert topics == new_topics

2022-06-01 06:04:08,349 - BERTopic - Reduced dimensionality
2022-06-01 06:04:08,605 - BERTopic - Predicted clusters


In [ ]:
topic_model.visualize_topics()

In [ ]:
fig = topic_model.visualize_topics()
fig.write_html("sample_data/file.html")

In [ ]:
topic_model.visualize_barchart(top_n_topics =  len(topic_model.topics) )